# Обучение нейрона с помощью функции потерь LogLoss

<h3 style="text-align: center;"><b>Нейрон с сигмоидой</b></h3>

Снова рассмотрим нейрон с сигмоидой, то есть $$f(x) = \sigma(x)=\frac{1}{1+e^{-x}}$$ 

Ранее мы установили, что **обучение нейрона с сигмоидой с квадратичной функцией потерь**:  

$$MSE(w, x) = \frac{1}{2n}\sum_{i=1}^{n} (\hat{y_i} - y_i)^2 = \frac{1}{2n}\sum_{i=1}^{n} (\sigma(w \cdot x_i) - y_i)^2$$    

где $w \cdot x_i$ - скалярное произведение, а $\sigma(w \cdot x_i) =\frac{1}{1+e^{-w \cdot x_i}} $ - сигмоида -- **неэффективно**, то есть мы увидели, что даже за большое количество итераций нейрон предсказывает плохо.

Давайте ещё раз взглянем на формулу для градиентного спуска от функции потерь $MSE$ по весам нейрона:

$$ \frac{\partial MSE}{\partial w} = \frac{1}{n} X^T (\sigma(w \cdot X) - y)\sigma(w \cdot X)(1 - \sigma(w \cdot X))$$

А теперь смотрим на график сигмоиды:

<img src="https://cdn-images-1.medium.com/max/1200/1*IDAnCFoeXqWL7F4u9MJMtA.png" width=500px height=350px>

**Её значения: числа от 0 до 1.**

Если получше проанализировать формулу, то теперь можно заметить, что, поскольку сигмоида принимает значения между 0 и 1 (а значит (1-$\sigma$) тоже принимает значения от 0 до 1), то мы умножаем $X^T$ на столбец $(\sigma(w \cdot X) - y)$ из чисел от -1 до 1, а потом ещё на столбцы $\sigma(w \cdot X)$ и $(1 - \sigma(w \cdot X))$ из чисел от 0 до 1. Таким образом в лучшем случае $\frac{\partial{Loss}}{\partial{w}}$ будет столбцом из чисел, порядок которых максимум 0.01 (в среднем, понятно, что если сигмоида выдаёт все 0, то будет 0, если все 1, то тоже 0). После этого мы умножаем на шаг градиентного спуска, который обычно порядка 0.001 или 0.1 максимум. То есть мы вычитаем из весов числа порядка ~0.0001. Медленновато спускаемся, не правда ли? Это называют **проблемой затухающих градиентов**.

Чтобы избежать эту проблему в задачах классификации, в которых моделью является нейрон с сигмоидной функцией активации, предсказывающий "вероятности" принадлженостей к классамиспользуют **LogLoss**:  

$$J(\hat{y}, y) = -\frac{1}{n} \sum_{i=1}^n y_i \log(\hat{y_i}) + (1 - y_i) \log(1 - \hat{y_i}) = -\frac{1}{n} \sum_{i=1}^n y_i \log(\sigma(w \cdot x_i)) + (1 - y_i) \log(1 - \sigma(w \cdot x_i))$$

где, как и прежде, $y$ - столбец $(n, 1)$ из истинных значений классов, а $\hat{y}$ - столбец $(n, 1)$ из предсказаний нейрона.

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd

Отметим, что сейчас речь идёт именно о **бинарной классификации (на два класса)**, в многоклассовой классификации используется функция потерь под названием *кросс-энтропия*, которая является обобщением LogLoss'а на случай нескольких классов.

Почему же теперь всё будет лучше? Раньше была проблема умножения маленьких чисел в градиенте. Давайте посмотрим, что теперь:

* Для веса $w_j$:

$$ \frac{\partial Loss}{\partial w_j} = 
-\frac{1}{n} \sum_{i=1}^n \left(\frac{y_i}{\sigma(w \cdot x_i)} - \frac{1 - y_i}{1 - \sigma(w \cdot x_i)}\right)(\sigma(w \cdot x_i))_{w_j}' = -\frac{1}{n} \sum_{i=1}^n \left(\frac{y_i}{\sigma(w \cdot x_i)} - \frac{1 - y_i}{1 - \sigma(w \cdot x_i)}\right)\sigma(w \cdot x_i)(1 - \sigma(w \cdot x_i))x_{ij} = $$
$$-\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{ij}$$

* Градиент $Loss$'а по вектору весов -- это вектор, $j$-ая компонента которого равна $\frac{\partial Loss}{\partial w_j}$ (помним, что весов всего $m$):

$$\begin{align}
    \frac{\partial Loss}{\partial w} &= \begin{bmatrix}
           -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{i1} \\
           -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{i2} \\
           \vdots \\
           -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i)\right)x_{im}
         \end{bmatrix}
\end{align}=\frac{1}{n} X^T \left(\hat{y} - y\right)$$

По аналогии с $w_j$ выведите формулу для свободного члена (bias'а) $b$ (*hint*: можно считать, что при нём есть признак $x_{i0}=1$ на всех $i$):

$$ \frac{\partial Loss}{\partial b_j} = 
-\frac{1}{n} \sum_{i=1}^n \left(\frac{y_i}{\sigma(w \cdot x_i + b_i)} - \frac{1 - y_i}{1 - \sigma(w \cdot x_i  + b_i)}\right)(\sigma(w \cdot x_i  + b_i))_{b_j}' = -\frac{1}{n} \sum_{i=1}^n \left(\frac{y_i}{\sigma(w \cdot x_i  + b_i )} - \frac{1 - y_i}{1 - \sigma(w \cdot x_i  + b_i)}\right)\sigma(w \cdot x_i  + b_i)(1 - \sigma(w \cdot x_i  + b_i)) = $$
$$-\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i  + b_i)\right)$$

$$\begin{align}
    \frac{\partial Loss}{\partial b} &= \begin{bmatrix}
           -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i + b_i)\right) \\
           -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i + b_i)\right) \\
           \vdots \\
           -\frac{1}{n} \sum_{i=1}^n \left(y_i - \sigma(w \cdot x_i + b_i)\right)
         \end{bmatrix}
\end{align}=\frac{1}{n} \left(\hat{y} - y\right)$$

Получили новое правило для обновления $w$ и $b$. 

In [ ]:
class Neuron:
    
    def __init__(self, w=None, b=0):
        """
        :param: w -- вектор весов
        :param: b -- смещение
        """
        # пока что мы не знаем размер матрицы X, а значит не знаем, сколько будет весов
        self.w = w
        self.b = b
    

    def sigmoid(self, x):
        linear = torch.mm(x,self.w) + self.b
        return 1/(1+torch.exp(linear))

    def activate(self, x):
        return self.sigmoid(x)
    
    def loss(self, y_pred, y):
        return -(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()
    
        
    def forward_pass(self, X):
        """
        Эта функция рассчитывает ответ нейрона при предъявлении набора объектов
        :param: X -- матрица объектов размера (n, m), каждая строка - отдельный объект
        :return: вектор размера (n, 1) из нулей и единиц с ответами перцептрона 
        """
        out = self.activate(X)
        return out
        
    def backward_pass(self, X, y, y_pred, learning_rate):
        """
        Обновляет значения весов нейрона в соответствие с этим объектом
        :param: X -- матрица объектов размера (n, m)
                y -- вектор правильных ответов размера (n, 1)
                learning_rate - "скорость обучения" (символ alpha в формулах выше)
        В этом методе ничего возвращать не нужно, только правильно поменять веса
        с помощью градиентного спуска.
        """
        # тут нужно обновить веса по формулам, написанным выше

        self.w = self.w - learning_rate*(1/X.shape[0])*torch.mm(X.T,(y - y_pred))
        self.b = self.b - learning_rate*(y - y_pred).mean()
        
    def fit(self, X, y, num_epochs=5000,learning_rate=0.1):
        """
        Спускаемся в минимум
        :param: X -- матрица объектов размера (n, m)
                y -- вектор правильных ответов размера (n, 1)
                num_epochs -- количество итераций обучения
        :return: J_values -- вектор значений функции потерь
        """
        torch.seed(41)
        self.w = torch.rand((X.shape[1],1)).double()  # столбец (m, 1)
        self.b = 0  # смещение
        loss_values = []  # значения функции потерь на различных итерациях обновления весов
        
        for i in range(num_epochs):
            # предсказания с текущими весами
            y_pred = self.forward_pass(X)
            # считаем функцию потерь с текущими весами
            loss_values.append(self.loss(y_pred, y))
            # обновляем веса по формуле градиентного спуска
            self.backward_pass(X, y, y_pred,learning_rate)

        return loss_values

    def predict(self,X):
        preds = torch.zeros((X.shape[0],1))
        out = self.activate(X)
        return out

<h3 style="text-align: center;"><b>Тестирование</b></h3>

Протестируем нейрон, обученный с новой функцией потерь, на тех же данных, что и в предыдущем ноутбуке:

**Проверка forward_pass()**

In [ ]:
import torch

In [ ]:
w = torch.tensor([1., 2.]).reshape(2, 1)
b = 2.
X = torch.tensor([[1., 3.],
              [2., 4.],
              [-1., -3.2]])

neuron = Neuron(w, b)
y_pred = neuron.forward_pass(X)
print ("My y_pred = " + str(y_pred))

My y_pred = tensor([[1.2339e-04],
        [6.1442e-06],
        [9.9550e-01]])


**Проверка backward_pass()**

In [ ]:
y = torch.tensor([1, 0, 1]).reshape(3, 1)

In [ ]:
neuron.backward_pass(X, y, y_pred,learning_rate = 1e-03)

print ("w = " + str(neuron.w))
print ("b = " + str(neuron.b))

w = tensor([[0.9997],
        [1.9990]])
b = tensor(1.9997)


Проверьте на наборах данных "яблоки и груши" и "голос".

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df_train, df_test = train_test_split(pd.read_csv("/content/apples_pears.csv"),random_state = 41)

In [ ]:
X_train = torch.from_numpy(df_train.iloc[:,:-1].to_numpy())
y_train = torch.from_numpy(df_train.iloc[:,-1].to_numpy()).reshape(-1,1)
X_test = torch.from_numpy(df_test.iloc[:,:-1].to_numpy())
y_test = torch.from_numpy(df_test.iloc[:, -1].to_numpy()).reshape(-1,1)

In [ ]:
model = Neuron()
losses = model.fit(X_train,y_train,num_epochs= 10**4)

preds = torch.where(model.predict(X_test) > 0.50, 1,0)

accuracy_score(preds,y_test.reshape(-1))
print(f"Accuracy with tune weight model {accuracy_score(preds,y_test.reshape(-1))}")

Accuracy with tune weight model 0.952


In [ ]:
model = Neuron()
model.w = torch.rand((X_train.shape[1],1)).double()
preds = torch.where(model.predict(X_test) > 0.50, 1,0)

print(f"Accuracy with random initialization model {accuracy_score(preds,y_test.reshape(-1))}")


Accuracy with random initialization model 0.384


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(max_iter = 10**4)
model.fit(X_train.numpy(),y_train.numpy().reshape(-1))
print(f"Accuracy with box model {accuracy_score(model.predict(X_test),y_test.reshape(-1))}")

Accuracy with box model 0.944


In [ ]:
df_train, df_test = train_test_split(pd.read_csv("/content/voice.csv"))

In [ ]:
X_train = torch.from_numpy(df_train.iloc[:,:-1].to_numpy())
y_train = torch.from_numpy(np.where(df_train.iloc[:,-1].to_numpy() == "female", 1,0)).reshape(-1,1)
X_test = torch.from_numpy(df_test.iloc[:,:-1].to_numpy())
y_test = torch.from_numpy(np.where(df_test.iloc[:,-1].to_numpy() == "female", 1,0)).reshape(-1,1)

In [ ]:
model = Neuron()
model.w = torch.rand((X_train.shape[1],1)).double()
preds = torch.where(model.predict(X_test) > 0.50, 1,0)

print(f"Accuracy with random initialization model {accuracy_score(preds,y_test.reshape(-1))}")

Accuracy with random initialization model 0.4962121212121212


In [ ]:
model = Neuron()
losses = model.fit(X_train,y_train, num_epochs= 40000, learning_rate=5e-03)

preds = torch.where(model.predict(X_test) >= 0.50, 1,0)

print(f"Accuracy with tune weight model {accuracy_score(preds,y_test.reshape(-1))}")

Accuracy with tune weight model 0.7878787878787878


In [ ]:
print(f"Weight in model = {str(model.w)}, bias in {str(model.b)}")

Weight in model = tensor([[-0.0518],
        [ 0.6156],
        [ 0.1048],
        [-1.0717],
        [ 0.6273],
        [ 1.4545],
        [-1.2041],
        [ 0.1239],
        [ 1.9607],
        [ 1.7206],
        [-0.1005],
        [ 0.2946],
        [-2.0645],
        [ 0.8079],
        [-0.2717],
        [-0.3224],
        [-0.2182],
        [-0.3066],
        [ 0.2693],
        [ 0.2172]], dtype=torch.float64), bias in tensor(0.1734, dtype=torch.float64)


In [ ]:
model = LogisticRegression(max_iter = 10**5)
model.fit(X_train.numpy(),y_train.numpy().reshape(-1))
print(f"Accuracy with box model {accuracy_score(model.predict(X_test),y_test.reshape(-1))}")

Accuracy with box model 0.9191919191919192
